In [13]:
import os
import sys
sys.path.append("..")

In [14]:
from global_config import *

sys.path.append(
    os.path.join(MMSEG_DIR_PATH, "tools", "analysis_tools")
)
sys.path[-1]

'/home/featurize/work/AI6126project1/mmsegmentation/tools/analysis_tools'

In [15]:
from get_flops import inference
from mmengine.logging import MMLogger

logger = MMLogger.get_instance(name='MMLogger', log_level="ERROR")

/home/featurize/work/.local/lib/python3.7/site-packages/mmengine/utils/manager.py:114: UserWarning: <class 'mmengine.logging.logger.MMLogger'> instance named of MMLogger has been created, the method `get_instance` should not accept any other arguments
  f'{cls} instance named of {name} has been created, '


In [16]:
# from pathlib import Path
# mmseg_configs_root_path = Path(MMSEG_MODEL_CONFIG_DIR_PATH)

# subfolders = [p.name for p in mmseg_configs_root_path.iterdir() if p.is_dir()]
# verified_subfolders = ['resnest', 'icnet', 'gcnet', 'hrnet', 'stdc', 'setr', 'ddrnet', 'twins', 'emanet', 'dnlnet', 'dpt', 'fastscnn', 'deeplabv3plus', 'dmnet', 'mobilenet_v3', 'dsdl', 'knet', 'apcnet', 'segnext', 'fastfcn', 'isanet', 'upernet', 'sem_fpn', 'psanet', 'unet', 'cgnet', 'bisenetv2', 'erfnet', 'pidnet']
# removed_subfolders = ["maskformer", "mask2former", "point_rend", "convnext", "beit", "poolformer", "ocrnet", "vpd", "_base_"]

# remaining_subfolders = list(set(subfolders) - set(verified_subfolders)-set(removed_subfolders))
# subfolders, remaining_subfolders

## model config file path

In [17]:
model = "fastscnn.pth"
model = "mobilnetv3_lraspp.pth"
model =  "bisenetv2_fcn.pth"

config_path = CKPT_CONFIG_LOOKUPS[model]["config"]
checkpoint_path = CKPT_CONFIG_LOOKUPS[model]["checkpoint"]
shape = 512
config_path, checkpoint_path

('/home/featurize/work/AI6126project1/out/bisenetv2_fcn/20250326_094633/vis_data/config.py',
 '/home/featurize/work/AI6126project1/out/bisenetv2_fcn/20250326_094633/best_mFscore_iter_16000.pth')

In [18]:
import argparse
def parse_args(arg_list):
    parser = argparse.ArgumentParser(description='Get the FLOPs of a segmentor')
    parser.add_argument('config', help='train config file path')
    parser.add_argument(
        '--shape',
        type=int,
        nargs='+',
        default=[2048, 1024],
        help='input image size'
    )
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        help='override some settings in the used config'
    )

    return parser.parse_args(arg_list)

## nparams result

In [19]:
arg_list = [
    config_path,
    "--shape",
    str(shape),
    # str(shape),
]
args = parse_args(arg_list)
#args
result = inference(args, logger)
params = result["params"]
flops = result["flops"]
params, flops

('1.792M', '6.024G')

In [20]:
from mmseg.apis import init_model
model = init_model(config_path, checkpoint_path, device="cuda:0")

/home/featurize/work/AI6126project1/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/featurize/work/AI6126project1/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


Loads checkpoint by local backend from path: /home/featurize/work/AI6126project1/out/bisenetv2_fcn/20250326_094633/best_mFscore_iter_16000.pth


In [21]:
from torchsummary import summary
summary(model, (3, 512, 512), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             864
       BatchNorm2d-2         [-1, 32, 256, 256]              64
              ReLU-3         [-1, 32, 256, 256]               0
        ConvModule-4         [-1, 32, 256, 256]               0
            Conv2d-5         [-1, 32, 256, 256]           9,216
       BatchNorm2d-6         [-1, 32, 256, 256]              64
              ReLU-7         [-1, 32, 256, 256]               0
        ConvModule-8         [-1, 32, 256, 256]               0
            Conv2d-9         [-1, 64, 128, 128]          18,432
      BatchNorm2d-10         [-1, 64, 128, 128]             128
             ReLU-11         [-1, 64, 128, 128]               0
       ConvModule-12         [-1, 64, 128, 128]               0
           Conv2d-13         [-1, 64, 128, 128]          36,864
      BatchNorm2d-14         [-1, 64, 1